# Banc de radiologie

Ci-dessous l'image du générateur de rayons X utilisé dans le banc de travaux pratiques.

![title](img/ImageInstallationRXcrop.png)

---

# Simulation de spectre de rayons X

Le code de simulation permet de calculer le spectre de rayons X générés par un tube X fonctionnant à une tension donnée. La mesure du spectre se fait à 1 mètre de la source dans l'air, mais une plaque de matériau peut être insérée. Sans plaque, le spectre est normalisé pour donné une dose de **1mGy** dans l'air à **1m**. Il faut ainsi préciser :
- la haute tension U d’accélération des électrons,
- le type de matériau pour la plaque, 
- l'épaisseur de cette plaque.

Certaines grandeurs associées aux spectres sont calculées en sortie notamment
- l’énergie moyenne des photons du spectre
- le nombre de photons mesuré
- la couche de demi-atténuation (en mm d’Al),

In [31]:
from xpecgen import xpecgen as xg
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from tabletext import to_text
import numpy as np
import xraylib as xrl
import xraylib_np as xrl_np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as tic
import matplotlib as mpl
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

xrl.XRayInit()
xrl.SetErrorMessages(0)

def f(E0,Mat_Z,Mat_X):
    s2=xg.calculate_spectrum(E0,12,3,100,epsrel=0.5,monitor=None,z=74)
    mu_Al=xg.get_mu(13)
    mu_Mat=xg.get_mu(Mat_Z)
    s2.attenuate(0.12,mu_Al) #1.2 mm of Al
    s2.attenuate(100,xg.get_mu("air")) #100 cm of Air
    fluence_to_dose=xg.get_fluence_to_dose()
    s2.set_norm(value=1.,weight=fluence_to_dose)
    s2.attenuate(0.1*Mat_X,mu_Mat)
    Nr_Photons = "%.4g" % (s2.get_norm())
    Average_Energy = "%.2f keV" % (s2.get_norm(lambda x:x)/s2.get_norm())
    Dose = "%.3g mGy" % (s2.get_norm(fluence_to_dose))    
    hvl_Al=s2.hvl(0.5,fluence_to_dose,mu_Al)
    HVL1_Al = "%.2f mm (Al)" % (10*hvl_Al)
    a = [["Dose à 1m", Dose],["Nombre total de photons", Nr_Photons],["Énergie moyenne des photons",Average_Energy],["Couche de Demie-Atténuation", HVL1_Al]]
    print(to_text(a))
    (x2,y2) = s2.get_points()
    plt.figure(1,dpi=150)
    mpl.rcParams.update({'font.size': 6})
    axMW = plt.subplot(111)
    axMW.plot(x2,y2)
    axMW.set_xlim(3,E0)
    axMW.set_ylim(0,)
    plt.xlabel("Énergie [keV]")
    plt.ylabel("Nombre de photons par [keV·cm²·mGy] @ 1m")
    axMW.grid(which='major', axis='x', linewidth=0.5, linestyle='-', color='0.75')
    axMW.grid(which='minor', axis='x', linewidth=0.2, linestyle='-', color='0.85')
    axMW.grid(which='major', axis='y', linewidth=0.5, linestyle='-', color='0.75')
    axMW.grid(which='minor', axis='y', linewidth=0.2, linestyle='-', color='0.85')
    axMW.xaxis.set_major_formatter(tic.FormatStrFormatter("%d"))
    axMW.yaxis.set_major_formatter(tic.FormatStrFormatter("%.2g"))
    axMW.xaxis.set_minor_locator(AutoMinorLocator())
    axMW.yaxis.set_minor_locator(AutoMinorLocator())
    axMW.grid(True)
    plt.show()

interact_manual(f, 
                E0 = widgets.IntText(value=100,step=1,description="Haute tension (kV) :",disabled=False,style={'description_width': 'initial'}),
                Mat_Z = widgets.Dropdown(options=[('Al',13),('Cu',29),('Pb',82)],value=29,layout={'width': 'max-content'},description='Matériau :',style={'description_width': 'initial'}),
                Mat_X = widgets.FloatText(value=0.,step=0.01,description="Épaisseur (mm) :",disabled=False,style={'description_width': 'initial'}));


interactive(children=(IntText(value=100, description='Haute tension (kV) :', style=DescriptionStyle(descriptio…

---

# Coefficient d'atténuation linéique

Le code ci-dessous permet de calculer le coefficient d'atténuation linéique $\mu$
de 2 matériaux en précisant leur formule chimique et leur densité

> Pour mémoire, la loi d'atténuation du nombre de photons directement transmis -- loi dite de Beer-Lambert -- est
> $$ N_{dt} = N_0 \exp{-\mu x} $$
> où $x$ représente l'épaisseur de matériau


In [26]:
def mus(mat1="Mg",rho1=1.74,mat2="C2F4",rho2=2.25):
    energy_range = np.arange(5.,800., 0.1, dtype=np.double)
    mu1 = [xrl.CS_Total_CP(mat1, E) * float(rho1) for E in energy_range]
    mu2 = [xrl.CS_Total_CP(mat2, E) * float(rho2) for E in energy_range]
    plt.figure(1,dpi=150)
    mpl.rcParams.update({'font.size': 6})
    axMW = plt.subplot(111)
    axMW.plot(energy_range, mu1,color="black",linewidth=2.,linestyle="-",label=mat1)
    axMW.plot(energy_range, mu2,color="red",linewidth=1.5,linestyle="-",label=mat2)
    axMW.set_xscale('log')
    axMW.set_yscale('log')
    axMW.set_xlim(5.,500.)
    #axMW.set_ylim(0.05,1000.)
    plt.legend(loc='center right', frameon=True)
    plt.xlabel("Énergie (keV)")
    plt.ylabel("Coefficient d'atténuation linéique (cm$^{-1}$)")
    axMW.grid(which='major', axis='x', linewidth=0.5, linestyle='-', color='0.75')
    axMW.grid(which='minor', axis='x', linewidth=0.2, linestyle='-', color='0.85')
    axMW.grid(which='major', axis='y', linewidth=0.5, linestyle='-', color='0.75')
    axMW.grid(which='minor', axis='y', linewidth=0.2, linestyle='-', color='0.85')
    axMW.xaxis.set_major_formatter(tic.FormatStrFormatter("%d"))
    axMW.grid(True)
    plt.show()

interact_manual(mus,
                mat1=widgets.Text(value="Mg",description='Matériau 1 :',disabled=False, style={'description_width': 'initial'}),
                rho1=widgets.FloatText(value=1.74,step=0.01,description="Densité 1 :",disabled=False, style={'description_width': 'initial'}),
                mat2=widgets.Text(value="C2F4",description='Matériau 2 :',disabled=False, style={'description_width': 'initial'}),
                rho2=widgets.FloatText(value=2.25,step=0.01,description="Densité 2 :",disabled=False, style={'description_width': 'initial'}));

interactive(children=(Text(value='Mg', description='Matériau 1 :', style=DescriptionStyle(description_width='i…